In [2]:
from utils import *
%matplotlib inline

# Fast, portable neural networks with gluon `.hybridize()`


<center><img src="support/fast.gif" width=300><center>

The tutorials we saw so far adopt the *imperative*, or define-by-run, programming paradigm. 
It might not even occur to you to give a name to this style of programming 
because it's how we always write Python programs. 

Take for example a prototypical program written below in pseudo-Python.
We grab some input arrays, we compute upon them to produce some intermediate values,
and finally we produce the result that we actually care about.

## Imperative / Symbolic Design

## Imperative Pseudofunction
```
def our_function(A, B, C, D):
    # Compute some intermediate values
    E = basic_function1(A, B)
    F = basic_function2(C, D)
    
    # Produce the thing you really care about
    G = basic_function3(E, F)
    return G
    
# Load up some data
W = some_stuff()
X = some_stuff()
Y = some_stuff()
Z = some_stuff()
    
result = our_function(W, X, Y, Z)
```

## Symbolic Pseudofunction

```
# Placeholders to stand in for real data
A = placeholder() 
B = placeholder()
C = placeholder()
D = placeholder()

# Compute some intermediate values
E = symbolic_function1(A, B)
F = symbolic_function2(C, D)
    
# Produce the thing you really care about
G = symbolic_function3(E, F)
    
our_function = library.compile(inputs=[A, B, C, D], outputs=[G])   
    
# Load up some data
W = some_stuff()
X = some_stuff()
Y = some_stuff()
Z = some_stuff()
    
result = our_function(W, X, Y, Z)
```

## Tradeoffs 

### Imperative Programs Tend to be More Flexible
* familiar style means you can use typical constructs.
* faster debugging, means you get to try out more ideas.
* the catch is that imperative programs are *comparatively* slow

### Symbolic Programs Tend to be More Efficient
* memory efficiency via reuse for intermediate results
* speed optimizations via operator folding
* the catch is the tricky indirection of working with placeholders

## Getting the best of both worlds with MXNet Gluon's HybridBlocks


**All of MXNet's predefined layers are HybridBlocks.** This means that any network consisting entirely of predefined MXNet layers can be compiled and run at much faster speeds by calling ``.hybridize()``.

In [3]:
import mxnet as mx
from mxnet.gluon import nn
from mxnet import nd

## HybridSequential

In [4]:
def get_net():
    # construct a MLP
    net = nn.HybridSequential()
    with net.name_scope():
        net.add(nn.Dense(256, activation="relu"))
        net.add(nn.Dense(128, activation="relu"))
        net.add(nn.Dense(2))
    # initialize the parameters
    net.collect_params().initialize()
    return net

# forward
x = nd.random_normal(shape=(1, 512))
net = get_net()
print('=== net(x) ==={}'.format(net(x)))

=== net(x) ===
[[0.08827585 0.0050519 ]]
<NDArray 1x2 @cpu(0)>


Once we compute the gradient of $f(x)$ with respect to $x$, we’ll need a place to store it. In MXNet, we can tell an NDArray that we plan to store a gradient by invoking its `attach_grad` method.

In [5]:
net.hybridize()
print('=== net(x) ==={}'.format(net(x)))

=== net(x) ===
[[0.08827585 0.0050519 ]]
<NDArray 1x2 @cpu(0)>


## Performance
Compare the performance before and after hybridizing 
by measuring the time it takes to make 1000 forward passes through the network.

In [6]:
from time import time
def bench(net, x):
    mx.nd.waitall()
    start = time()
    for i in range(1000):
        y = net(x)
    mx.nd.waitall()
    return time() - start

In [7]:
net = get_net()
print('Before hybridizing: %.4f sec'%(bench(net, x)))
net.hybridize()
print('After hybridizing: %.4f sec'%(bench(net, x)))

Before hybridizing: 0.3493 sec
After hybridizing: 0.1783 sec


## Get the symbolic program
If we feed the network with a Symbol placeholder, then the corresponding symbolic program will be returned.

In [8]:
from mxnet import sym
x = sym.var('data')
print('=== input data holder ===')
print(x)

y = net(x)
print('\n=== the symbolic program of net===')
print(y)

y_json = y.tojson()
print('\n=== the corresponding json definition===')
print(y_json)

=== input data holder ===
<Symbol data>

=== the symbolic program of net===
<Symbol hybridsequential1_dense2_fwd>

=== the corresponding json definition===
{
  "nodes": [
    {
      "op": "null", 
      "name": "data", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential1_dense0_weight", 
      "attrs": {
        "__dtype__": "0", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256, 0)", 
        "__storage_type__": "0", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential1_dense0_bias", 
      "attrs": {
        "__dtype__": "0", 
        "__init__": "zeros", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256,)", 
        "__storage_type__": "0", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "FullyConnected", 
      "name": "hybridsequential1_dense0_fwd", 
      "attrs": {
        "flatten": "True", 
        "no_bias": "False

## Save symbolic model
Now we can save both the program and parameters onto disk, so that it can be loaded later not only in Python, but in all other supported languages, such as C++, R, and Scala, as well.

In [9]:
net.export('my_model', epoch=0)

### let's dive deeper into how `hybridize` works.
* Recall, Gluon networks are composed of Blocks each of which subclass `gluon.Block`
* For hybrid networks, we have `gluon.HybridBlock`
* To define a `HybridBlock`, we have to define a`hybrid_forward` function:

## HybridBlock

In [10]:
from mxnet import gluon

class Net(gluon.HybridBlock):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            self.fc1 = nn.Dense(256)
            self.fc2 = nn.Dense(128)
            self.fc3 = nn.Dense(2)

    def hybrid_forward(self, F, x):
        # F is a function space that depends on the type of x
        # If x's type is NDArray, then F will be mxnet.nd
        # If x's type is Symbol, then F will be mxnet.sym
        print('type(x): {}, F: {}'.format(
                type(x).__name__, F.__name__))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

The hybrid_forward function takes an additional input, F, which stands for a backend. This exploits one awesome feature of MXNet. MXNet has both a symbolic API (mxnet.symbol) and an imperative API (mxnet.ndarray).

In [11]:
net = Net()
net.collect_params().initialize()
x = nd.random_normal(shape=(1, 512))
print('=== 1st forward ===')
y = net(x)
print('=== 2nd forward ===')
y = net(x)

=== 1st forward ===
type(x): NDArray, F: mxnet.ndarray
=== 2nd forward ===
type(x): NDArray, F: mxnet.ndarray


In [9]:
net.hybridize()
print('=== 1st forward ===')
y = net(x)
print('=== 2nd forward ===')
y = net(x)

=== 1st forward ===
type(x): Symbol, F: mxnet.symbol
=== 2nd forward ===


## Conclusion

Through `HybridSequental` and `HybridBlock`, we can convert an imperative program into a symbolic program by calling `hybridize`. 